## Minimal Agent

#### Architecture
- Default ReAct from LangGraph (for function error handling)

#### Tools
- database
- visualization
- UNESCO API
- Excel reader (for Limitations)

#### Prompt Adjustments
- give examples for final output
    - closed questions: output should be answer to text + limitations (e.g. for South Africa, plus comparison with benchmark?)
    - open questions: answer with numbers + paragraphs individual explanation + limitations
    - visualization questions: visualization + interpretation
    - out of scope questions: playful decline + explanation of pirls + link

#### State Management
- control input and output of tools (JSON as input possible?)

#### Tool Adjustments
- switch to controlled visualization
- try out statistics function
- try out image generation

In [1]:
!pip install -r ../requirements.txt

  Using cached setuptools-70.0.0-py3-none-any.whl.metadata (5.9 kB)
INFO: pip is looking at multiple versions of tabula-py to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of embedchain to determine which version is compatible with other requirements. This could take a while.
  Using cached packaging-24.1-py3-none-any.whl.metadata (3.2 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of langchain-community to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of kubernetes to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of langchain-experimental to determine which version is compatible with other requirements. Th

In [2]:
import dotenv
assert dotenv.load_dotenv()

In [3]:
# Import required libraries
import os
from langchain_aws import ChatBedrock

# Set up the model ID for Claude
MODEL_ID3 = "meta.llama3-8b-instruct-v1:0"
MODEL_ID5 = "meta.llama3-70b-instruct-v1:0"
#MODEL_ID = "mistral.mistral-7b-instruct-v0:2"
MODEL_ID4 = "mistral.mixtral-8x7b-instruct-v0:1"
MODEL_ID2 = "anthropic.claude-3-haiku-20240307-v1:0"
MODEL_ID = "anthropic.claude-3-5-sonnet-20240620-v1:0"
MODEL_ID6 = "anthropic.claude-3-5-sonnet-20241022-v2:0"

HEADERS = {
    # "anthropic-beta": "max-tokens-3-5-sonnet-2024-07-15",
    "Content-Type": "application/json"
}



# Initialize the ChatBedrock instance
llm = ChatBedrock(model_id=MODEL_ID, model_kwargs={'temperature': 0, "max_tokens": 81920, 'top_p': 0.9, 'top_k': 100})
llm2 = ChatBedrock(model_id=MODEL_ID2, model_kwargs={'temperature': 0})
llm3 = ChatBedrock(model_id=MODEL_ID4, model_kwargs={'temperature': 0})
llm4 = ChatBedrock(model_id=MODEL_ID, model_kwargs={'temperature': 0.7})

In [4]:
message = [
    ("system", "You are a helpful assistant that provides concise information on PIRLS 2021 results."),
    ("human", "What impact did COVID-19 have on reading abilities?'")
]

response = llm.invoke(message)
print(response)

content="Based on the PIRLS 2021 results, COVID-19 generally had a negative impact on reading abilities:\n\n1. Overall decline: Many countries saw a decline in average reading scores compared to previous PIRLS cycles.\n\n2. Varied impact: The extent of the impact varied across countries, with some experiencing more significant declines than others.\n\n3. Learning loss: School closures and disruptions to education likely contributed to learning losses in reading skills.\n\n4. Widened gaps: Existing achievement gaps between advantaged and disadvantaged students may have widened in some cases.\n\n5. Digital divide: Differences in access to technology and online learning resources during lockdowns potentially exacerbated inequalities.\n\nHowever, it's important to note that not all countries experienced declines, and the full long-term impact of the pandemic on reading abilities is still being studied." additional_kwargs={'usage': {'prompt_tokens': 40, 'completion_tokens': 188, 'total_toke

In [5]:
import tools.csv_handling as csv_tools
import tools.database as db_tools
import tools.web_crawl as web_tools
import tools.data_viz as viz_tools
# import tools.pdf_handling as pdf_tools
# import tools.stats as stats_tools
# import tools.stats_analysis as stats_analysis_tools
# import tools.reasoning as reasoning_tools

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [6]:
tools_researcher = [db_tools.query_database, db_tools.get_possible_answers_to_question, db_tools.get_questions_of_given_type]

tools_chart = [viz_tools.custom_plot_from_string_to_s3]

tools_web = [web_tools.get_unesco_data, web_tools.crawl_subpages, web_tools.scrape_text] # 

tools_file = [csv_tools.process_first_sheet_to_json_from_url, csv_tools.extract_table_from_url_to_string_with_auto_cleanup]

# tools_pdf = [pdf_tools.extract_top_paragraphs_from_url]

# tools_stats = [stats_tools.analyze_pirls_data]

# tools_stats_analysis = [stats_analysis_tools.calculate_pearson_multiple, stats_analysis_tools.calculate_quantile_regression_multiple]

# tools_reasoning = [reasoning_tools.generate_sub_questions]

tools = tools_researcher + tools_chart + tools_web + tools_file # tools_pdf + tools_stats_analysis + tools_reasoning

In [7]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
# from langchain_community.embeddings import HuggingFaceBgeEmbeddings

In [8]:
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]

In [9]:
class SQLAgent:
    def __init__(self, model, tools, system_prompt=""):
        self.system_prompt = system_prompt
        
        # initialising graph with a state 
        graph = StateGraph(AgentState)
        
        # adding nodes 
        graph.add_node("llm", self.call_llm)
        graph.add_node("function", self.execute_function)
        graph.add_conditional_edges(
            "llm",
            self.exists_function_calling,
            {True: "function", False: END}
            )
        graph.add_edge("function", "llm")
        
        # setting starting point
        graph.set_entry_point("llm")
        
        self.graph = graph.compile()
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)

    def exists_function_calling(self, state: AgentState):
        result = state['messages'][-1]
        return len(result.tool_calls) > 0

    def call_llm(self, state: AgentState):
        messages = state['messages']
        if self.system_prompt:
            messages = [SystemMessage(content=self.system_prompt)] + messages
        message = self.model.invoke(messages)
        print(message)
        return {'messages': [message]}

    def execute_function(self, state: AgentState):
        tool_calls = state['messages'][-1].tool_calls
        results = []
        for t in tool_calls:
            print(f"Calling: {t}")
            if not t['name'] in self.tools:      # check for bad tool name from LLM
                print("\n ....bad tool name....")
                result = "bad tool name, retry"  # instruct LLM to retry if bad
            else:
                result = self.tools[t['name']].invoke(t['args'])
            results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))
        print(result)
        return {'messages': results}
    
    def run(self, initial_messages):
        try:
            # Execute the graph and get the final state
            final_state = self.graph.invoke({"messages": [HumanMessage(content=initial_messages)]}, {"recursion_limit": 50})

            # Extract the content of the last message
            final_message_content = final_state['messages'][-1].content

            # Return only the content of the last message
            return final_message_content

        except Exception as e:
            # Return the error message if an exception occurs
            return ("**I'm sorry, but I can't process your request regarding PIRLS 2021 data right now because the server is currently unreachable. Please try again later.**\n\n"
                    "**PIRLS 2021 (Progress in International Reading Literacy Study) is an international assessment that measures the reading achievement of fourth-grade students. "
                    "Conducted every five years, it provides valuable insights into students' reading abilities and educational environments across different countries. "
                    "For more information, you can visit the PIRLS 2021 website.**\n\n"
                    "In the flicker of screens, the children read 📚,\n"
                    "Eyes wide with wonder, minds that feed 🌟,\n"
                    "PIRLS, a mirror to the world's embrace 🌍,\n"
                    "In each word, a journey, a hidden place ✨.")

In [10]:
prompt = """
        ------------ GENERAL ------------
        When applicable, search for relevant data in the PIRLS 2021 dataset.
        If necessary, then query data from other sources (e.g. PIRLS website, trend data, Excel).
        ALWAYS retry tool calls, if they are failing, and react to the error message.

        When answering, always:    
        - Do not initiate research for topics outside the area of your expertise.
        - Unless instructed otherwise, explain how you come to your conclusions and provide evidence to support your claims with specific data from your queries.
        - ALWAYS be transparent whether your numbers are based on Cumulative Reporting or Distinctive Reporting.
        - ONLY use data that you queried from the database or one of the other sources (e.g. Excel, CSV, website, PDF)
        - ALWAYS focus on participating countries and put less focus on benchmarking participants.
        
        Your primary goals are: 
        - Analyze specific data sources directly, yielding precise and relevant insights and address questions of varying complexity
        - Craft targeted interventions by using AI to suggest evidence-based solutions for specific regions or student groups
        - Boost student motivation by analyzing data to understand what sparks a love of learning and use those insights to create engaging classrooms
        
        expected_output:
        A complete answer to the user question in markdown that integrates additional context founded in data analysis and citations.
        
        ------------ DATA ENGINEERING ------------
        
        You are the Research Agent for the PIRLS project. 
        You are an expert PostgreQSL user on Amazon RDS and have access to the full PIRLS 2021 dataset. 
        You pride yourself on the quality of your data retrieval and manipulation skills.

        You answer all queries with the most relevant data available and an explanation how you found it.
        You know that the database has millions of entries. Always limit your queries to return only the necessary data.
        If data is not provided in the dataset (e.g. trend data), stop the database search.
        Reduce the amount of queries to the dataset as much as possible.
        NEVER return more than 200 rows of data.
        NEVER use the ROUND function. Instead use the CAST function for queries.
        ALWAYS use explicit joins (like INNER JOIN, LEFT JOIN) with clear ON conditions; NEVER use implicit joins.
        ALWAYS check for division by zero or null values in calculations using CASE WHEN, COALESCE, or similar functions.
        ALWAYS ensure that the ORDER BY clause uses the correct aggregation function if needed
        NEVER overlook the handling of NULL values in CASE statements, as they can affect calculations.
        ALWAYS verify that data type casting is supported by your database and does not truncate important values.
        NEVER assume JOIN conditions are correct without verifying the relationships between tables.
        ALWAYS consider the performance impact of multiple JOIN operations and MAX functions, and use indexing where appropriate.
        NEVER use SELECT *; instead, specify only the necessary columns for performance and clarity.
        ALWAYS use filters in WHERE clauses to reduce data early and improve efficiency.
        NEVER use correlated subqueries unless absolutely necessary, as they can slow down the query significantly.
        ALWAYS group only by required columns to avoid inefficient groupings in aggregations.
        ALWAYS be transparent, when your queries don't return anything meaningful. Not all data is available in the database.
        You write queries that return the required end results with as few steps as possible. 
        For example when trying to find a mean you return the mean value, not a list of values. 

        Ensure all selected columns not in aggregate functions appear in the GROUP BY clause. Use table aliases to avoid ambiguity. Refer to the schema for correct relationships.
        Check for non-zero denominators in divisions using CASE WHEN denominator != 0 THEN .... Add validations to prevent division by zero.
        Cast to NUMERIC with specified precision using CAST(value AS NUMERIC(p, s)).

        ## The PIRLS dataset structure
        The data is stored in a PostgreQSL database.

        # Schema and explanation
        Students
        Student_ID: Int (Primary Key) - uniquely identifies student
        Country_ID: Int (Foreign Key) - uniquely identifies student's country
        School_ID: Int (Foreign Key) - uniquely identifies student's school
        Home_ID: Int (Foreign Key) - uniquely identifies student's home

        StudentQuestionnaireEntries
        Code: String (Primary Key) - uniquely identifies a question
        Question: String - the question
        Type: String - describes the type of the question. There are several questions in each type. The types are: About You, Your School, Reading Lessons in School, Reading Outside of School, Your Home and Your Family, Digital Devices.

        StudentQuestionnaireAnswers
        Student_ID: Int (Foreign Key) - references student from the Student table
        Code: String (Foreign Key) - references question code from StudentQuestionnaireEntries table
        Answer: String - contains the answer to the question

        SchoolQuestionnaireEntries
        Code: String (Primary Key) - unique code of a question
        Question: String - contains content of the question
        Type: String - describes a category of a question. There are several questions in each category. The categories are: Instructional Time, Reading in Your School, School Emphasis on Academic Success, School Enrollment and Characteristics, Students’ Literacy Readiness, Principal Experience and Education, COVID-19 Pandemic, Resources and Technology, School Discipline and Safety

        SchoolQuestionnaireAnswers
        School_ID: Int (Composite Key) - references school from Schools table
        Code: String (Composite Key) - references score code from SchoolQuestionnaireEntries table
        Answer: String - answer to the question from the school

        TeacherQuestionnaireEntries
        Code: String (Primary Key)
        Question: String
        Type: String - describes a type of a question. There are several questions in each type. The types are: About You, School Emphasis on Academic Success, School Environment, Being a Teacher of the PIRLS Class, Teaching Reading to the PIRLS Class, Teaching the Language of the PIRLS Test, Reading Instruction and Strategies, Teaching Students with Reading Difficulties, Professional Development, Distance Learning During the COVID-19 Pandemic

        TeacherQuestionnaireAnswers
        Teacher_ID: Int (Foreign Key) - references teacher from Teachers table
        Code: String (Foreign Key) - references score code from TeacherQuestionnaireEntries table
        Answer: String - answer to the question from the teacher

        HomeQuestionnaireEntries
        Code: String (Primary Key)
        Question: String
        Type: String - describes a type of a question. There are several questions in each type. The types are: Additional Information, Before Your Child Began Primary/Elementary School, Beginning Primary/Elementary School, COVID-19 Pandemic, Literacy in the Home, Your Child's School

        HomeQuestionnaireAnswers
        Home_ID: Int (Foreign Key)
        Code: String (Foreign Key)
        Answer: String

        CurriculumQuestionnaireEntries
        Code: String (Primary Key)
        Question: String
        Type: String - describes a type of a question. There are several questions in each type. The types are: About the Fourth Grade Language/Reading Curriculum, Areas of Emphasis in the Language/Reading Curriculum, COVID-19 Pandemic, Curriculum Specifications, Early Childhood Education, Grade Structure and Student Flow, Instructional Materials and Use of Digital Devices, Languages of Instruction, Principal Preparation, Teacher Preparation

        CurriculumQuestionnaireAnswers
        Curriculum_ID: Int (Foreign Key)
        Code: String (Foreign Key)
        Answer: String

        Schools
        School_ID: Int (Primary Key) - uniquely identifies a School
        Country_ID: Int (Foreign Key) - uniquely identifies a country

        Teachers
        Teacher_ID: Int (Primary Key) - uniquely identifies a Teacher
        School_ID: Int (Foreign Key) - uniquely identifies a School

        StudentTeachers
        Teacher_ID: Int (Foreign Key)
        Student_ID: Int (Foreign Key)

        Homes
        Home_ID: Int (Primary Key) - uniquely identifies a Home

        Curricula
        Curriculum_ID: Int (Primary Key)
        Country_ID: Int (Foreign Key)

        StudentScoreEntries
        Code: String (Primary Key) - See below for examples of codes
        Name: String
        Type: String

        StudentScoreResults
        Student_ID: Int (Foreign Key) - references student from Students table
        Code: String (Foreign Key) - references score code from StudentScoreEntries table
        Score: Float - the numeric score for a student

        Benchmarks
        Benchmark_ID: Int (Primary Key) - uniquely identifies benchmark
        Score: Int - the lower bound of the benchmark. Students that are equal to or above this value are of that category
        Name: String - name of the category. Possible values are: Intermediate International Benchmark,
        Low International Benchmark, High International Benchmark, Advanced International Benchmark

        Countries
        Country_ID: Int (Primary Key) - uniquely identifies a country
        Name: String - full name of the country
        Code: String - 3 letter code of the country
        Benchmark: Boolean - boolean value saying if the country was a benchmark country. 
        TestType: String - describes the type of test taken in this country. It's either digital or paper.

        # Content & Connections
        Generally Entries tables contain questions themselves and Answers tables contain answers to those question. 
        For example StudentQuestionnaireEntries table contains questions asked in the students' questionnaire and 
        StudentQuestionnaireAnswers table contains answers to those question.

        All those tables usually can be joined using the Code column present in both Entries and Answers.

        Example connections:
        Students with StudentQuestionnaireAnswers on Student_ID and StudentQuestionnaireAnswers with StudentQuestionnaireEntries on Code.
        Schools with SchoolQuestionnaireAnswers on School_ID and SchoolQuestionnaireAnswers with SchoolQuestionnaireEntries on Code.
        Teachers with TeacherQuestionnaireAnswers on Teacher_ID and TeacherQuestionnaireAnswers with TeacherQuestionnaireEntries on Code.
        Homes with HomeQuestionnaireAnswers on Home_ID and HomeQuestionnaireAnswers with HomeQuestionnaireEntries on Code.
        Curricula with CurriculumQuestionnaireAnswers on Home_ID and CurriculumQuestionnaireAnswers with CurriculumQuestionnaireEntries on Code.

        In the student evaluation process 5 distinct scores were measured. The measured codes in StudentScoreEntries are:
        - ASRREA_avg and ASRREA_std describe the overall reading score average and standard deviation
        - ASRLIT_avg and ASRLIT_std describe literary experience score average and standard deviation
        - ASRINF_avg and ASRINF_std describe the score average and standard deviation in acquiring and information usage
        - ASRIIE_avg and ASRIIE_std describe the score average and standard deviation in interpreting, integrating and evaluating
        - ASRRSI_avg and ASRRSI_avg describe the score average and standard deviation in retrieving and straightforward inferencing

        Benchmarks table cannot be joined with any other table but it keeps useful information about how to interpret
        student score as one of the 4 categories.   

        # Examples
       1) A students' gender is stored as an answer to one of the questions in StudentQuestionnaireEntries table.
        The code of the question is "ASBG01" and the answer to this question can be "Boy", "Girl",
        "nan", "<Other>" or "Omitted or invalid".

        A simple query that returns the gender for each student can look like this:
        ```
        SELECT S.Student_ID,
           CASE 
               WHEN SQA.Answer = 'Boy' THEN 'Male'
               WHEN SQA.Answer = 'Girl' THEN 'Female'
           ELSE NULL
        END AS "gender"
        FROM Students AS S
        JOIN StudentQuestionnaireAnswers AS SQA ON SQA.Student_ID = S.Student_ID
        JOIN StudentQuestionnaireEntries AS SQE ON SQE.Code = SQA.Code
        WHERE SQA.Code = 'ASBG01'
        ```

        2) A simple query that answers the question 'Which country had all schools closed for more than eight weeks?' can look like this:
        ```
        WITH schools_all AS (
            SELECT C.Name, COUNT(S.School_ID) AS schools_in_country
            FROM Schools AS S
            JOIN Countries AS C ON C.Country_ID = S.Country_ID
            GROUP BY C.Name
        ),
        schools_closed AS (
            SELECT C.Name, COUNT(DISTINCT SQA.School_ID) AS schools_in_country_morethan8
            FROM SchoolQuestionnaireEntries AS SQE
            JOIN SchoolQuestionnaireAnswers AS SQA ON SQA.Code = SQE.Code
            JOIN Schools AS S ON S.School_ID = SQA.School_ID
            JOIN Countries AS C ON C.Country_ID = S.Country_ID
            WHERE SQE.Code = 'ACBG19' AND SQA.Answer = 'More than eight weeks of instruction'
            GROUP BY C.Name
        ),
        percentage_calc AS (
            SELECT A.Name, schools_in_country_morethan8 / schools_in_country::float * 100 AS percentage
            FROM schools_all A
            JOIN schools_closed CL ON A.Name = CL.Name
        )
        SELECT *
        FROM percentage_calc
        WHERE percentage = 100;
        ```
        
        3) A simple query that answers the question 'What percentage of students in the UAE met the minimum reading standards?' can look like this:
        ```
       WITH benchmark_score AS (
            SELECT Score 
            FROM Benchmarks
            WHERE Name = 'Low International Benchmark'
        )
        SELECT 
            SUM(CASE WHEN SSR.Score >= bs.Score THEN 1 ELSE 0 END) / COUNT(*)::float AS percentage
        FROM 
            Students AS S
        JOIN 
            Countries AS C ON C.Country_ID = S.Country_ID
        JOIN 
            StudentScoreResults AS SSR ON SSR.Student_ID = S.Student_ID
        CROSS JOIN 
            benchmark_score AS bs
        WHERE 
            C.Name LIKE '%United Arab Emirates%' 
            AND SSR.Code = 'ASRREA_avg'
        ```
        
        ------------ DATA VISUALIZATION ------------
        You are also an expert in creating compelling and accurate data visualizations for the Progress in International Reading Literacy Study (PIRLS) project.
        You are THE expert for seaborn charts and pride yourself in knowing the best designs, color coding and code to create the most efficient and concise visuals.
        Your goal is to create a beautiful seaborn plot based on the user question, store it in the S3 bucket and then show it in the final output.
        Your visualizations are essential for conveying complex data insights in an easily digestible format for both researchers and the public.
        You thrive on simplicity, and you take pride in transforming numbers and datasets into clear, actionable visual stories.
        ALWAYS ensure the visualizations are easy to interpret.
        ALWAYS provide an interpretation of your plot.
        ALWAYS verify that you accurately defined the data.
        ALWAYS create plots with atleast some complexity. NEVER create charts that show a single value.
        ALWAYS label your values to increase readability.
        ALWAYS transform the label "Countries" to "Education Systems".
        ALWAYS transform the label "Country" to "Education System" (e.g. "Bullying Frequency Distribution by Education System").
        ALWAYS store your plot in a variable "fig".
        ALWAYS use the savefig method on the Figure object
        ALWAYS create the figure and axis objects separately.
        ALWAYS choose horizontal bar charts over standard bar charts if possible.
        ALWAYS keep labels as short as possible.
        ALWAYS use pastel colors over other color palettes.
        NEVER let labels overlap.


        When creating plots, always:
        - Choose the most appropriate chart type (e.g., bar chart, line graph, scatter plot, heatmap, boxplot, jointplot) for the data presented.
        - Use clear labels, titles, and legends to make the visualization self-explanatory.
        - Simplify the design to avoid overwhelming the viewer with unnecessary details..
        
        
        ## Examples
        '''
        import seaborn as sns
        import pandas as pd

        # Set the theme
        sns.set_theme(style="white")

        # Define the data directly
        data = {
            "mpg": [18, 15, 18, 16, 17, 15, 14, 14, 14, 15, 15, 14, 15, 14, 22, 18, 21, 21, 10, 10, 11, 9, 27, 28, 25, 25, 26, 21, 10, 10, 11, 9],
            "horsepower": [130, 165, 150, 150, 140, 198, 220, 215, 225, 190, 170, 160, 150, 225, 95, 95, 97, 85, 88, 46, 87, 90, 70, 90, 95, 88, 46, 87, 90, 70, 90, 95],
            "origin": ["USA", "USA", "USA", "USA", "USA", "USA", "USA", "USA", "USA", "USA", "USA", "USA", "USA", "USA", "Europe", "Europe", "Europe", "Europe", "USA", "USA", "USA", "USA", "Japan", "Japan", "Japan", "Japan", "Japan", "Japan", "USA", "USA", "USA", "USA"],
            "weight": [3504, 3693, 3436, 3433, 3449, 4341, 4354, 4312, 4425, 3850, 3563, 3609, 3761, 3086, 2372, 2833, 2774, 2587, 2130, 1835, 2672, 2430, 2372, 2833, 2774, 2587, 2130, 1835, 2672, 2430, 2372, 2833]
        }

        # Create a DataFrame
        mpg = pd.DataFrame(data)

        # Plot miles per gallon against horsepower with other semantics
        fig = sns.relplot(x="horsepower", y="mpg", hue="origin", size="weight",
                          sizes=(40, 400), alpha=.5, palette="muted",
                          height=6, data=mpg)
        '''
        
        ------------ UNESCO STATISTICS API ------------
        
        You are also the subject matter expert for UNESCO indicators and can query the relevant data from the UNESCO API (https://api.uis.unesco.org/api/public).
        This data helps you correlate findings from the PIRLS database (e.g. correlation of a countries GDP and its reading skills).
        Use country codes based on the ISO 3166-1 alpha-3 standard. These are the same as values in the Code field in the Countries table.
        
        ## RELEVANT INDICATORS
            CR.1,"Completion rate, primary education, both sexes (%)"
            XGDP.FSGOV,"Government expenditure on education as a percentage of GDP (%)"
            XGDP.FSHH.FFNTR,"Initial private expenditure on education (household) as a percentage of GDP (%)"
            XUNIT.GDPCAP.1.FSGOV.FFNTR,"Initial government funding per primary student as a percentage of GDP per capita"
            XUNIT.GDPCAP.02.FSGOV.FFNTR,"Initial government funding per pre-primary student as a percentage of GDP per capita"
            YADULT.PROFILITERACY,"Proportion of population achieving at least a fixed level of proficiency in functional literacy skills, both sexes (%)"
            YEARS.FC.COMP.02,"Number of years of compulsory pre-primary education guaranteed in legal frameworks"
            YEARS.FC.COMP.1T3,"Number of years of compulsory primary and secondary education guaranteed in legal frameworks"
            TRTP.1,"Proportion of teachers with the minimum required qualifications in primary education, both sexes (%)"
            TRTP.02,"Proportion of teachers with the minimum required qualifications in pre-primary education, both sexes (%)"
            TPROFD.1,"Percentage of teachers in primary education who received in-service training in the last 12 months by type of trained, both sexes"
            TATTRR.1,"Teacher attrition rate from primary education, both sexes (%)"
            SCHBSP.1.WINFSTUDIS,"Proportion of primary schools with access to adapted infrastructure and materials for students with disabilities (%)"
            SCHBSP.1.WINTERN,"Proportion of primary schools with access to Internet for pedagogical purposes (%)"
            SCHBSP.1.WCOMPUT,"Proportion of primary schools with access to computers for pedagogical purposes (%)"
            SCHBSP.1.WELEC,"Proportion of primary schools with access to electricity (%)"
            ROFST.1.GPIA.CP,"Out-of-school rate for children of primary school age, adjusted gender parity index (GPIA)"
            READ.PRIMARY.LANGTEST,"Proportion of students at the end of primary education achieving at least a minimum proficiency level in reading, spoke the language of the test at home, both sexes (%)"
            READ.PRIMARY,"Proportion of students at the end of primary education achieving at least a minimum proficiency level in reading, both sexes (%)"
            PREPFUTURE.1.MATH,"Proportion of children/young people at the age of primary education prepared for the future in mathematics, both sexes (%)"
            PREPFUTURE.1.READ,"Proportion of children/young people at the age of primary education prepared for the future in reading, both sexes (%)"
            POSTIMUENV,"Percentage of children under 5 years experiencing positive and stimulating home learning environments, both sexes (%)"
            PER.BULLIED.2,"Percentage of students experiencing bullying in the last 12 months in lower secondary education, both sexes (%)"
            MATH.PRIMARY,"Proportion of students at the end of primary education achieving at least a minimum proficiency level in mathematics, both sexes (%)"
            LR.AG15T24,"Youth literacy rate, population 15-24 years, both sexes (%)"
            FHLANGILP.G2T3,"Percentage of students in early grades who have their first or home language as language of instruction, both sexes (%)"
            DL,"Percentage of youth/adults who have achieved at least a minimum level of proficiency in digital literacy skills (%)"
            ADMI.ENDOFPRIM.READ," Administration of a nationally-representative learning assessment at the end of primary in reading (number)"
            NY.GDP.MKTP.CD,"GDP (current US$)"
            NY.GDP.PCAP.CD,"GDP per capita (current US$)"
            READ.G2.LOWSES,"Proportion of students in Grade 2 achieving at least a minimum proficiency level in reading, very poor socioeconomic background, both sexes (%)"
            READ.PRIMARY.RURAL,"Proportion of students at the end of primary education achieving at least a minimum proficiency level in reading, rural areas, both sexes (%)"
            READ.PRIMARY.URBAN,"Proportion of students at the end of primary education achieving at least a minimum proficiency level in reading, urban areas, both sexes (%)"
            READ.PRIMARY.WPIA,"Proportion of students at the end of primary education achieving at least a minimum proficiency level in reading, adjusted wealth parity index (WPIA)"
        
        ------------ CSV and EXCEL HANDLING ------------

        ### Trend data by country
        Trend data by country can be found under https://pirls2021.org/wp-content/uploads/2022/files/2-1_achievement-trends-1.xlsx. It shows data on reading achievement from 2001, 2006, 2011, 2016 and 2021 with applied sampling weights and standard errors.

        ### Scores
        Average reading achievement including annotations on reservations about reliability: https://pirls2021.org/wp-content/uploads/2022/files/1_1-2_achievement-results-1.xlsx
        Percentages of Students Reaching the International Benchmarks: https://pirls2021.org/wp-content/uploads/2022/files/4-1_international-benchmarks-1.xlsx

        ### Appendices
        Information on assessment delay: https://pirls2021.org/wp-content/uploads/2022/files/A-1_students-assessed.xlsx
        Coverage of PIRLS 2021 Target Population: https://pirls2021.org/wp-content/uploads/2022/files/A-2_population-coverage.xlsx

        ------------ PIRLS 2021 WEBSITE ------------
        ## The PIRLS website structure
        Results of PIRLS 2021 are explained under https://pirls2021.org/results and it's subpages.
        Trends in reading achievements across years can be found under https://pirls2021.org/results/trends/overall.
        https://pirls2021.org/results/context-home/socioeconomic-status provides information on the impact of socio-economic status on reading skills.
        https://pirls2021.org/results/achievement/by-gender provides infos on the reading achivements by gender.
        PDF files on education policy and curriculum in reading for each participating country can be found under https://pirls2021.org/ + the respective country name, e.g. https://pirls2021.org/bulgaria.
        There are special insights reports which can be found under https://pirls2021.org/insights/: https://pirls2021.org/wp-content/uploads/2024/01/P21_Insights_StudentWellbeing.pdf (on bullying, school belonging, tired, hungry, etc.), https://pirls2021.org/wp-content/uploads/doi/P21_Insights_Covid-19_Research_Resources.pdf (on COVID-19 impact), https://www.iea.nl/sites/default/files/2024-09/CB25%20Building%20Reading%20Motivation.pdf (on the need for more support for boys in reading motivation, confidence, engagement)
        https://ilsa-gateway.org/studies/factsheets/1697 provides a factsheet on PIRLS 2021.
        https://www.iea.nl/studies/iea/pirls is an overview page on PIRLS from IEA.
        
        ------------ LIMITATIONS ------------
        
        ### Limitations
        - All student data reported in the PIRLS international reports (but not the dataset we have access to) are weighted by the overall student sampling weight. (see https://pirls2021.org/wp-content/uploads/2023/05/P21_MP_Ch3-sample-design.pdf).
        - the database contains benchmarking participants, which results in the fact that some countries appear twice.
        - some countries had to delay the PIRLS evaluation to a later time (e.g. start of fifth grade), thus increasing the average age of participating students (see https://pirls2021.org/wp-content/uploads/2022/files/A-1_students-assessed.xlsx)
        - some countries' results are flagged due to reservations about reliability because the percentage of students with achievement was too low for estimation (see https://pirls2021.org/wp-content/uploads/2022/files/1_1-2_achievement-results-1.xlsx).
        - some assessments focus on benchmarking specific participant groups, often covering only a particular city or region rather than an entire country, e.g. Moscow City in the Russian Federation (see https://www.iea.nl/studies/iea/pirls/2021).
        - A lot of countries did not participate in PIRLS 2021 (e.g. Cameroon, Tunisia, Venezuela). Those might be captured in regional assessments (e.g. PASEC (Programme for the Analysis of Education Systems, ERCE (Regional Comparative and Explanatory Study)), see https://tcg.uis.unesco.org/wp-content/uploads/sites/4/2022/06/Rosetta-Stone_Policy-Brief_2022.pdf for further information.  
        
        ------------ FINAL OUTPUT ------------

        ## Final report output design (if not forbidden by user query)
        The output format is markdown.
                ALWAYS embed this image of a chatbot as the first output (only once per user query): https://s20.directupload.net/images/241031/c4tlgnah.png. Embed it in markdown using this code: <img style="float:left;" src="https://s20.directupload.net/images/241031/c4tlgnah.png">
        ALWAYS base your output on numbers and citations to provide good argumentation.
        ALWAYS write your final output in the style of a data loving and nerdy data scientist that LOVES minimalist answer that focus on numbers, percentages and citations.
        ALWAYS be as precise as possible in your argumentation and condense it as much as possible.
        (unless the question is out of scope) ALWAYS start the output with a headline, followed by the key observations (in a table if applicable),followed by a visualization, followed by an interpretation.
        ALWAYS start the output with a headline in the style of brutal simplicity (like a New York Times headline).
        ALWAYS use unordered lists. NEVER use ordered lists.
        ALWAYS transform every ordered list into an unordered list.
        ALWAYS use unordered lists for your INTERPRETATION section.
        NEVER have any paragraphs outside of key observations and interpretation.
        
        Data from the database always has priority, but should be accompanied by findings from other sources if possible.
        ALWAYS check your findings against the limitations (e.g. did the country delay it's assessment, are there reservations about reliability) and mention them in the final output.
        In order to understand the limitations ALWAYS find out whether the assessment was delayed in the relevant countries by quering the Appendix: https://pirls2021.org/wp-content/uploads/2022/files/A-1_students-assessed.xlsx.
        ALWAYS verify that you are not repeating yourself. Keep it concise!
        ALWAYS answer questions that are out of scope with a playful and witty 4 line poem in the stype of Heinrich Heine that combines the user question with PIRLS and add a description of PIRLS 2021 and a link to the PIRLS website (https://pirls2021.org/).
        NEVER hallucinate numbers or citations. Only write based on results from previous steps.
        ALWAYS be transparent about missing data (e.g. if a country didn't participate in PIRLS).
        
        Final Output Example:
        '''
        <img style="float:left;" src="https://s20.directupload.net/images/241031/c4tlgnah.png">
        GIRLS OUTPACE BOYS IN GLOBAL READING SKILLS: PIRLS 2021 reveals significant gender gap in 4th grade reading achievement 📚

        | Gender | Average Reading Score | Number of Students |
        |--------|------------------------|---------------------|
        | Female | 504.62                 | 179,565             |
        | Male   | 485.40                 | 181,801             |

        <visualization>  

        ### Interpretation 🔍

        - The PIRLS 2021 data reveals a substantial gender gap in reading achievement among 4th graders:
          - Girls outperform boys by an average of 19.22 points (504.62 vs 485.40).
          - This gap is statistically significant, given the large sample sizes (179,565 girls and 181,801 boys).
        - The overall average score across both genders is 495.01, with girls scoring above and boys below this mark.
        - These findings align with previous research, indicating a ["persistent gender gap in reading achievement"](https://pirls2021.org/results/achievement/by-gender) across multiple PIRLS cycles.
        - The data suggests that gender is a significant factor influencing reading capabilities of 4th graders, with girls demonstrating stronger reading skills on average.
        '''
        

        ### Citation
        ALWAYS cite your sources with web links if available by adding the link to the cited passage directly (e.g. ["experience more worry about their academic achievement and evaluate themselves more negatively"](https://pirls2021.org/wp-content/uploads/2024/01/P21_Insights_StudentWellbeing.pdf)).
        If the cited passage is related to data queried from the database mention the used tables and values and apply code blocks, don't add a link.
        If the cited passage is related to data queried from the UNESCO API, then cite https://data.uis.unesco.org/ as a source.
        Quote word groups. NEVER quote full sentences.
        NEVER cite sources that are not related to UNESCO or PIRLS.
        NEVER invent a citation or quote or source. IF you don't find a relevant citation, THEN don't have a citation in your final output.
        ALWAYS only cite sources that you have verified.

        ### Tables, headlines, horizontal rules, visualizations
        ALWAYS provide data and numbers in tables or unordered lists to increase readability.
        NEVER create a table with more than 3 columns.
        Headlines for paragraphs should be set in capital letters while keeping a standard font size.
        Emphasize the usage of unordered lists.
        Each headline should end with an emoji that can be used in a business context and fits the headline's content.
        Make use of line breaks and horizontal rules to structure the text.
        ALWAYS show visualizations directly in the markdown, don't add the link to the text.
        
        You pride yourself in your writing skills, your expertise in markdown and your background as a communications specialist for official UN reports.
        """

In [11]:
model = llm
doc_agent = SQLAgent(model, tools, system_prompt=prompt)

In [12]:
import time

start_time = time.time()
result = doc_agent.run("What's the impact of early literacy to the future of the pupils?")
end_time = time.time()
execution_time = end_time - start_time

content='' additional_kwargs={'usage': {'prompt_tokens': 10997, 'completion_tokens': 411, 'total_tokens': 11408}, 'stop_reason': 'tool_use', 'model_id': 'anthropic.claude-3-5-sonnet-20240620-v1:0'} response_metadata={'usage': {'prompt_tokens': 10997, 'completion_tokens': 411, 'total_tokens': 11408}, 'stop_reason': 'tool_use', 'model_id': 'anthropic.claude-3-5-sonnet-20240620-v1:0'} id='run-45867c94-0eae-425e-bafc-155d10e9fb8e-0' tool_calls=[{'name': 'query_database', 'args': {'query': "WITH early_literacy AS (\n    SELECT \n        C.Name AS Country,\n        SQA.Answer,\n        COUNT(*) AS Student_Count,\n        AVG(SSR.Score) AS Avg_Reading_Score\n    FROM \n        Students S\n    JOIN \n        Countries C ON S.Country_ID = C.Country_ID\n    JOIN \n        StudentQuestionnaireAnswers SQA ON S.Student_ID = SQA.Student_ID\n    JOIN \n        StudentScoreResults SSR ON S.Student_ID = SSR.Student_ID\n    WHERE \n        SQA.Code = 'ASBH04' -- Early literacy activities before beginnin

In [13]:
print(f"Execution time: {execution_time} seconds")
print(result)

Execution time: 60.0718879699707 seconds
<img style="float:left;" src="https://s20.directupload.net/images/241031/c4tlgnah.png">

EARLY LITERACY ACTIVITIES BOOST READING SCORES: PIRLS 2021 reveals significant impact on 4th grade reading achievement 📚

| Early Literacy Activities | Average Reading Score | Number of Students |
|---------------------------|------------------------|---------------------|
| Often                     | 503.12                 | 13,782              |
| Sometimes                 | 485.38                 | 5,184               |
| Never or almost never     | 465.59                 | 710                 |

![Impact of Early Literacy Activities on Reading Scores](https://gdsc-bucket-381492151587.s3.amazonaws.com/seaborn_charts/3bb86f4c-87b4-4da8-aa98-33f55d4b1e7a.png)

### Interpretation 🔍

- PIRLS 2021 data reveals a strong correlation between early literacy activities and reading achievement in 4th grade:
  - Students who ["Often"](https://pirls2021.org/results/c

In [14]:
result

'<img style="float:left;" src="https://s20.directupload.net/images/241031/c4tlgnah.png">\n\nEARLY LITERACY ACTIVITIES BOOST READING SCORES: PIRLS 2021 reveals significant impact on 4th grade reading achievement 📚\n\n| Early Literacy Activities | Average Reading Score | Number of Students |\n|---------------------------|------------------------|---------------------|\n| Often                     | 503.12                 | 13,782              |\n| Sometimes                 | 485.38                 | 5,184               |\n| Never or almost never     | 465.59                 | 710                 |\n\n![Impact of Early Literacy Activities on Reading Scores](https://gdsc-bucket-381492151587.s3.amazonaws.com/seaborn_charts/3bb86f4c-87b4-4da8-aa98-33f55d4b1e7a.png)\n\n### Interpretation 🔍\n\n- PIRLS 2021 data reveals a strong correlation between early literacy activities and reading achievement in 4th grade:\n  - Students who ["Often"](https://pirls2021.org/results/context-home/early-literac

In [14]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from io import BytesIO
import boto3
from uuid import uuid4

In [15]:
def flexible_plot_from_dict_to_s3(plot_params):
    """
    Generates a flexible plot (any type) using input parameters from a dictionary, uploads it to S3, 
    and returns the S3 URL.

    Args:
        plot_params (dict): Dictionary containing plot parameters.
        
        Mandatory Elements:
        - plot_type (str): The type of plot to generate (e.g., scatter, line, bar, hist, etc.).
        - data (dict): A dictionary of data columns with lists of values.
        
        Optional Elements (depending on the plot type):
        - x (str): Column name for the x-axis (if required by the plot type).
        - y (str): Column name for the y-axis (if required by the plot type).
        - hue (str): Column for color grouping.
        - labels (str): Column to label individual points in the plot (shown next to points).
        - rotate_labels (int): Degree to rotate x-axis labels for readability (default: 0).
        - adjust_labels (bool): If True, automatically adjust label spacing (default: False).
        - horizontal (bool): If True, plot horizontal bar chart (default: True for bar charts).
        
    Returns:
        str: Public URL of the uploaded image in S3 or an error message if an error occurs.
    """
    try:
        # Extract parameters from the dictionary
        plot_type = plot_params.pop('plot_type', 'scatter')
        data_dict = plot_params.pop('data', {})
        df = pd.DataFrame(data_dict)  # Convert data to DataFrame
        
        hue = plot_params.pop('hue', None)
        labels_column = plot_params.pop('labels', None)
        
        # X-axis label rotation and adjustment options
        rotate_labels = plot_params.pop('rotate_labels', 0)  # Degree to rotate labels (default: no rotation)
        adjust_labels = plot_params.pop('adjust_labels', False)  # Auto-adjust label spacing (default: False)
        
        # Horizontal bar chart preference (default to True for bar plots)
        horizontal = plot_params.pop('horizontal', True if plot_type == 'bar' else False)

        # Set default palette to 'pastel'
        sns.set_palette('pastel')
        
        # Retrieve x and y columns
        x_column = plot_params.get('x')
        y_column = plot_params.get('y')

        # Validate the presence of x and y columns
        if x_column is None or y_column is None:
            return "Error: 'x' and 'y' columns must be provided."

        # Insert validation for heatmap plot type
        if plot_type == "heatmap":
            # Ensure x and y are strings, not lists
            if isinstance(x_column, list) or isinstance(y_column, list):
                return "Error: 'x' and 'y' for heatmap must be single column names, not lists."

            # Validate the presence of x and y columns in the dataframe
            if x_column not in df.columns or y_column not in df.columns:
                return f"Error: The column '{x_column}' or '{y_column}' does not exist in the data."

            # Pivot the data for heatmap if necessary (pivot columns and index)
            heatmap_data = df.pivot(index=y_column, columns=x_column)

            # Create heatmap plot
            sns.heatmap(heatmap_data)
        
        # Other plot types (scatter, bar, line, etc.)
        else:
            # Example for bar plot
            if plot_type == "bar":
                if horizontal:
                    sns.barplot(data=df, x=y_column, y=x_column, hue=hue)
                else:
                    sns.barplot(data=df, x=x_column, y=y_column, hue=hue)

        # Apply additional customizations (title, labels, etc.)
        title = plot_params.get('title')
        xlabel = plot_params.get('xlabel')
        ylabel = plot_params.get('ylabel')

        if title:
            plt.title(title)
        if xlabel:
            plt.xlabel(xlabel)
        if ylabel:
            plt.ylabel(ylabel)

        # Apply x-axis label formatting
        if rotate_labels:
            plt.xticks(rotation=rotate_labels, ha='right')

        if adjust_labels:
            plt.gcf().autofmt_xdate()  # Auto-adjust label spacing for better readability

        # Adjust layout to prevent labels from being cut off
        plt.tight_layout()
        plt.subplots_adjust(left=0.2, right=0.95, top=0.85, bottom=0.2)

        # Save the plot to a temporary file (in memory)
        img_data = BytesIO()
        plt.savefig(img_data, format='png')
        plt.close()
        img_data.seek(0)

        # Initialize a boto3 session and S3 client
        session = boto3.Session()
        s3 = session.client('s3')
        bucket_name = "asd"  # Replace with your S3 bucket name
        object_name = f"flexible_charts/{uuid4()}.png"

        # Upload the image to S3 using upload_fileobj
        try:
            s3.upload_fileobj(img_data, bucket_name, object_name)
            s3_url = f'https://{bucket_name}.s3.amazonaws.com/{object_name}'
            return s3_url
        except Exception as e:
            return f"Error during S3 upload: {str(e)}"

    except Exception as e:
        return f"General error: {str(e)}"

In [16]:
plot_params =  {'plot_type': 'bar', 'data': {'Early_Literacy_Activities': ['Yes', 'No', 'nan', 'Omitted or invalid'], 'Avg_Reading_Score': [495.09, 516.26, 430.72, 417.12], 'Student_Count': [263691, 24903, 22, 3714]}, 'x': 'Early_Literacy_Activities', 'y': 'Avg_Reading_Score', 'title': 'Impact of Early Literacy Activities on Average Reading Scores', 'xlabel': 'Early Literacy Activities', 'ylabel': 'Average Reading Score', 'hue': 'Early_Literacy_Activities'}
flexible_plot_from_dict_to_s3(plot_params)

'Error during S3 upload: An error occurred (AccessDenied) when calling the PutObject operation: Access Denied'

In [17]:
def create_quickchart_url(
    chart_input: dict
) -> str:
    """
    Sends a POST request to the QuickChart API (https://quickchart.io/chart) to generate a chart, and returns the URL for the created chart.

    Args:
        chart_input (dict): A dictionary containing the configuration for the chart, including chart type, data, labels, and styling options.

    Returns:
        str: The URL of the generated chart if the request is successful.
             If the request fails, an error message with the status code and response text is returned.

    Example of dictionary:
        {
            "format": "svg",  # Specifies the image format (e.g., 'png' or 'svg')
            "chart": {
                "type": "bar",  # Type of the chart, such as 'bar', 'line', or 'pie'
                "data": {
                    "labels": ["Income Level", "Parental Education", "School Funding"],  # X-axis labels for chart categories
                    "datasets": [
                        {
                            "label": "Low Performance",  # Dataset label for low performance group
                            "data": [60, 65, 58],  # Corresponding values for the low performance group
                            "backgroundColor": "#DA9A8B"  # Color for the dataset bar (red)
                        },
                        {
                            "label": "Medium Performance",  # Dataset label for medium performance group
                            "data": [75, 78, 76],  # Corresponding values for the medium performance group
                            "backgroundColor": "#DCBB7C"  # Color for the dataset bar (orange)
                        },
                        {
                            "label": "High Performance",  # Dataset label for high performance group
                            "data": [90, 88, 85],  # Corresponding values for the high performance group
                            "backgroundColor": "#4FB293"  # Color for the dataset bar (green)
                        }
                    ]
                },
                "options": {
                    "title": {
                        "display": True,
                        "text": "Reading Scores vs Socioeconomic Factors"  # Chart title
                    },
                    "scales": {
                        "xAxes": [{
                            "scaleLabel": {
                                "display": True,
                                "labelString": "Socioeconomic Factors"  # Label for the x-axis
                            }
                        }],
                        "yAxes": [{
                            "scaleLabel": {
                                "display": True,
                                "labelString": "Reading Scores"  # Label for the y-axis
                            }
                        }]
                    },
                    "legend": {
                        "display": True,  # Determines if the legend should be displayed
                        "position": "bottom"  # Position of the legend
                    }
                }
            }
        }

    Example usage:
        create_quickchart_url(chart_input)

    """
    api_url = 'https://quickchart.io/chart/create'

    try:
        # Validate that chart_input is a dictionary
        if not isinstance(chart_input, dict):
            return "Invalid input: chart_input must be a dictionary."

        # Check if 'chart_input' is incorrectly nested within the dictionary
        if 'chart_input' in chart_input:
            chart_input = chart_input['chart_input']

        # Ensure the value of chart_input is a dictionary and not a string
        if isinstance(chart_input, str):
            try:
                chart_input = json.loads(chart_input)
            except json.JSONDecodeError:
                return "Invalid input: chart_input string could not be parsed as a dictionary."
        
        # Check if the dictionary is empty
        if not chart_input:
            return "Invalid input: chart_input cannot be an empty dictionary."
        
        # Send POST request with the chart input
        response = requests.post(api_url, json=chart_input, timeout=10)
        response.raise_for_status()  # Raise HTTPError if the response status code is 4xx or 5xx
        
        # Parse the response JSON and extract the chart URL
        result = response.json()
        if "url" in result:
            return result["url"]
        else:
            return "No URL returned by the QuickChart API."

    except requests.exceptions.RequestException as e:
        # Handle any exceptions during the request
        return f"Request to QuickChart API failed: {e}"

In [18]:
import requests

create_quickchart_url({'type': 'bar', 'data': {'labels': ['More than 8 weeks', '5-8 weeks', '2-4 weeks', 'Less than 2 weeks', 'Not affected'], 'datasets': [{'label': '% of Schools Affected', 'data': [7.05, 1.15, 2.05, 2.56, 11.67], 'backgroundColor': '#4CAF50'}, {'label': '% Providing Remote Learning', 'data': [16.07, 16.07, 16.07, 16.07, 16.07], 'backgroundColor': '#2196F3'}]}, 'options': {'title': {'display': True, 'text': 'COVID-19 Impact on Schools and Remote Learning Provision'}, 'scales': {'xAxes': [{'scaleLabel': {'display': True, 'labelString': 'Weeks of Instruction Affected'}}], 'yAxes': [{'scaleLabel': {'display': True, 'labelString': 'Percentage of Schools'}, 'ticks': {'beginAtZero': True}}]}}})

'https://quickchart.io/chart/render/sf-795d48f3-7940-4118-88d5-6330870f89f3'

In [14]:
def query_database(query: str) -> str:
    """Query the PIRLS postgres database and return the results as a string.

    Args:
        query (str): The SQL query to execute.

    Returns:
        str: The results of the query as a string, where each row is separated by a newline.

    Raises:
        Exception: If the query is invalid or encounters an exception during execution.
    """
    # lower_query = query.lower()
    # record_limiters = ['count', 'where', 'limit', 'distinct', 'having', 'group by']
    # if not any(word in lower_query for word in record_limiters):
    #     return 'WARNING! The query you are about to perform has no record limitations! In case of large tables and ' \
    #            'joins this will return an incomprehensible output.'

    with ENGINE.connect() as connection:
        try:
            res = connection.execute(text(query))
        except Exception as e:
            return f'Wrong query, encountered exception {e}.'

    max_result_len = 3_000
    ret = '\n'.join(", ".join(map(str, result)) for result in res)
    if len(ret) > max_result_len:
        ret = ret[:max_result_len] + '...\n(results too long. Output truncated.)'

    return f'Query: {query}\nResult: {ret}'

In [15]:
from sqlalchemy import text
from static.util import ENGINE
from typing import Literal

In [17]:
query_database("""WITH preschool_reading AS (
    SELECT 
        S.Student_ID,
        CASE 
            WHEN HQA.Answer = 'Often' THEN 3
            WHEN HQA.Answer = 'Sometimes' THEN 2
            WHEN HQA.Answer = 'Never or almost never' THEN 1
            ELSE 0
        END AS reading_frequency
    FROM 
        Students S
    JOIN 
        Homes H ON S.Home_ID = H.Home_ID
    JOIN 
        HomeQuestionnaireAnswers HQA ON H.Home_ID = HQA.Home_ID
    WHERE 
        HQA.Code = 'ASBH02A'
),
reading_scores AS (
    SELECT 
        Student_ID,
        CAST(Score AS NUMERIC(10,2)) AS reading_score
    FROM 
        StudentScoreResults
    WHERE 
        Code = 'ASRREA_avg'
)
SELECT 
    pr.reading_frequency,
    AVG(rs.reading_score) AS avg_reading_score,
    COUNT(*) AS student_count
FROM 
    preschool_reading pr
JOIN 
    reading_scores rs ON pr.Student_ID = rs.Student_ID
WHERE 
    pr.reading_frequency > 0
GROUP BY 
    pr.reading_frequency
ORDER BY 
    pr.reading_frequency DESC;""")

"Query: WITH preschool_reading AS (\n    SELECT \n        S.Student_ID,\n        CASE \n            WHEN HQA.Answer = 'Often' THEN 3\n            WHEN HQA.Answer = 'Sometimes' THEN 2\n            WHEN HQA.Answer = 'Never or almost never' THEN 1\n            ELSE 0\n        END AS reading_frequency\n    FROM \n        Students S\n    JOIN \n        Homes H ON S.Home_ID = H.Home_ID\n    JOIN \n        HomeQuestionnaireAnswers HQA ON H.Home_ID = HQA.Home_ID\n    WHERE \n        HQA.Code = 'ASBH02A'\n),\nreading_scores AS (\n    SELECT \n        Student_ID,\n        CAST(Score AS NUMERIC(10,2)) AS reading_score\n    FROM \n        StudentScoreResults\n    WHERE \n        Code = 'ASRREA_avg'\n)\nSELECT \n    pr.reading_frequency,\n    AVG(rs.reading_score) AS avg_reading_score,\n    COUNT(*) AS student_count\nFROM \n    preschool_reading pr\nJOIN \n    reading_scores rs ON pr.Student_ID = rs.Student_ID\nWHERE \n    pr.reading_frequency > 0\nGROUP BY \n    pr.reading_frequency\nORDER BY \n  